In [2]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('car_panorama/3.bmp')
x= image.shape[0]
y= image.shape[1]
print((x,y))


(400, 1280)


In [11]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('car_panorama/3.bmp')
height, width, _ = image.shape

# Define rotation parameters
angle = 45  # Rotation angle in degrees
theta = np.radians(angle)  # Convert angle to radians
d = width  # Distance of the image plane from the camera

# Define the perspective transformation matrix for rotation about the Y-axis
# This creates a 3D perspective effect
a = np.cos(theta)
b = np.sin(theta)

# Points of the source image (corners)
src_points = np.float32([[0, 0], [width, 0], [width, height], [0, height]])

# Points of the transformed image
dst_points = np.float32([
    [width/2 - d * b, 0],
    [width/2 + d * b, 0],
    [width/2 + d * b, height],
    [width/2 - d * b, height]
])

# Calculate the perspective transformation matrix
matrix = cv2.getPerspectiveTransform(src_points, dst_points)

# Apply the transformation
transformed_image = cv2.warpPerspective(image, matrix, (width, height))

# Save or display the transformed image
cv2.imshow('Transformed Image', transformed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [12]:
import cv2 as cv
import numpy as np
import subprocess
from pathlib import Path
import matplotlib.pyplot as plt

In [13]:
def run_superglue(script_path, input_pairs, input_dir, output_dir, superglue_weights="outdoor", resize_dimensions=-1):
    """
    Runs the SuperGlue matching script with specified parameters.
    """
    command = [
        "python3", script_path,
        "--input_pairs", input_pairs,
        "--input_dir", input_dir,
        "--output_dir", output_dir,
        "--superglue", superglue_weights,
        "--viz",
        "--resize", str(resize_dimensions),
        "--max_keypoints", "2048",
        "--nms_radius", "5",
        "--keypoint_threshold", "0.05",
        "--match_threshold", "0.9",
        "--resize_float"
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print("Error running SuperGlue:")
        print(result.stderr)
    return result

def loadNPZ(npz_file):
    """
    Loads the points from an NPZ file generated by SuperGlue.
    """
    data = np.load(npz_file)
    keypoints0 = data['keypoints0']
    keypoints1 = data['keypoints1']
    matches = data['matches']
    valid = matches > -1
    points1 = keypoints0[valid]
    points2 = keypoints1[matches[valid]]
    return points1, points2

def stitch_images(image1_path, image2_path, npz_path, output_path):
    """
    Stitches two images together using points from a SuperGlue-generated NPZ file.
    """
    im_left = cv.imread(image1_path, cv.IMREAD_ANYCOLOR)
    im_right = cv.imread(image2_path, cv.IMREAD_ANYCOLOR)
    point_set1, point_set2 = loadNPZ(npz_path)
    H, status = cv.findHomography(point_set1, point_set2, cv.RANSAC, 5.0)
    panorama = cv.warpPerspective(im_right, np.linalg.inv(H), (1500, 800))
    plt.imshow(panorama, cmap='gray', vmin = 0, vmax = 255)
    plt.show()
    panorama[0:im_left.shape[0], 0:im_left.shape[1]] = im_left
    pltSourceImages(image1_path,image2_path,point_set1, point_set2)
    plt.imshow(panorama, cmap='gray', vmin = 0, vmax = 255)
    plt.show()
    cv.imwrite(output_path, panorama)
    return output_path
def pltSourceImages(image1, image2, point_set1, point_set2):    
    im_left = cv.imread(image1,cv.IMREAD_ANYCOLOR)
    im_right = cv.imread(image2,cv.IMREAD_ANYCOLOR)
    
    # Marking the detected features on the two images.
    for point in point_set1.astype(np.int32):
        cv.circle(im_left, tuple(point), radius=8, color=(255, 255, 0), thickness=-1)

    for point in point_set2.astype(np.int32):
        cv.circle(im_right, tuple(point), radius=8, color=(255, 255, 0), thickness=-1)

    fig = plt.figure(figsize = (10,10))
    plt.subplot(121),plt.imshow(im_left, cmap='gray', vmin = 0, vmax = 255)
    plt.subplot(122),plt.imshow(im_right, cmap='gray', vmin = 0, vmax = 255)
    plt.show()

In [2]:
import cv2
import numpy as np
import os

# Define the folder path
input_folder = 'Stitch_with_perspective'
output_folder = input_folder+'/input'  # Save back to the original folder

# Define rotation parameters
angle = 32  # Rotation angle in degrees
theta = np.radians(angle)  # Convert angle to radians

# Loop through the images
for i in range(3, 21):  # Images numbered from 3 to 20
    # Construct the file path
    filename = f"{i}.bmp"
    input_path = os.path.join(input_folder, filename)

    # Load the image
    image = cv2.imread(input_path)
    if image is None:
        print(f"Image {filename} not found or couldn't be loaded.")
        continue

    # Get image dimensions
    height, width, _ = image.shape

    # Define transformation parameters
    d = width  # Distance of the image plane from the camera
    a = np.cos(theta)
    b = np.sin(theta)

    # Points of the source image (corners)
    src_points = np.float32([[0, 0], [width, 0], [width, height], [0, height]])

    # Points of the transformed image
    dst_points = np.float32([
        [width / 2 - d * b, 0],
        [width / 2 + d * b, 0],
        [width / 2 + d * b, height],
        [width / 2 - d * b, height]
    ])

    # Calculate the perspective transformation matrix
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Apply the transformation
    transformed_image = cv2.warpPerspective(image, matrix, (width, height))

    # Save the transformed image back to the original file
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, transformed_image)

    print(f"Processed and saved {filename}")

print("Processing complete.")


Processed and saved 3.bmp
Processed and saved 4.bmp
Processed and saved 5.bmp
Processed and saved 6.bmp
Processed and saved 7.bmp
Processed and saved 8.bmp
Processed and saved 9.bmp
Processed and saved 10.bmp
Processed and saved 11.bmp
Processed and saved 12.bmp
Processed and saved 13.bmp
Processed and saved 14.bmp
Processed and saved 15.bmp
Processed and saved 16.bmp
Processed and saved 17.bmp
Processed and saved 18.bmp
Processed and saved 19.bmp
Processed and saved 20.bmp
Processing complete.


In [3]:
import cv2 as cv
import numpy as np
import subprocess
from pathlib import Path
import matplotlib.pyplot as plt

def run_superglue(script_path, input_pairs, input_dir, output_dir, superglue_weights="outdoor", resize_dimensions=-1):
    """
    Runs the SuperGlue matching script with specified parameters.
    """
    command = [
        "python3", script_path,
        "--input_pairs", input_pairs,
        "--input_dir", input_dir,
        "--output_dir", output_dir,
        "--superglue", superglue_weights,
        "--viz",
        "--resize", str(resize_dimensions),
        "--max_keypoints", "2048",
        "--nms_radius", "5",
        "--keypoint_threshold", "0.05",
        "--match_threshold", "0.9",
        "--resize_float"
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print("Error running SuperGlue:")
        print(result.stderr)
    return result

def loadNPZ(npz_file):
    """
    Loads the points from an NPZ file generated by SuperGlue.
    """
    data = np.load(npz_file)
    keypoints0 = data['keypoints0']
    keypoints1 = data['keypoints1']
    matches = data['matches']
    valid = matches > -1
    points1 = keypoints0[valid]
    points2 = keypoints1[matches[valid]]
    return points1, points2

def stitch_images(image1_path, image2_path, npz_path, output_path):
    """
    Stitches two images together using points from a SuperGlue-generated NPZ file.
    """
    im_left = cv.imread(image1_path, cv.IMREAD_ANYCOLOR)
    im_right = cv.imread(image2_path, cv.IMREAD_ANYCOLOR)
    point_set1, point_set2 = loadNPZ(npz_path)
    H, status = cv.findHomography(point_set1, point_set2, cv.RANSAC, 5.0)
    rows1, cols1 = im_right.shape[:2]
    rows2, cols2 = im_left.shape[:2]
    points1 = np.float32([[0, 0], [0, rows1], [cols1, rows1], [cols1, 0]]).reshape(-1, 1, 2)
    points = np.float32([[0, 0], [0, rows2], [cols2, rows2], [cols2, 0]]).reshape(-1, 1, 2)
    points2 = cv.perspectiveTransform(points, H)
    points2_list = [point[0].tolist() for point in points2]
    list_of_points = np.concatenate((points1, points2), axis=0)

    [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() )
    [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() )

    H_translation = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]).dot(H)

    output_img = cv.warpPerspective(im_left, H_translation, (x_max - x_min, y_max - y_min))
    output_img[-y_min:rows1 + (-y_min), -x_min:cols1 + (-x_min)] = im_right
    input_pts = np.float32([[0,0], points2_list[1], points2_list[2], [cols1, 0]])
    output_pts = np.float32([[0, 0],
                        [0, points2_list[1][1]],
                        points2_list[2],
                        [points2_list[2][0], 0]])
    # Compute the perspective transform M
    M = cv.getPerspectiveTransform(input_pts, output_pts)
    final_img = cv.warpPerspective(output_img, M, (x_max - x_min, y_max - y_min))
    cv.imwrite(output_path, final_img)
    return output_path

def hierarchical_stitching(start, end, script_path, input_dir, output_dir):
    """
    Hierarchical stitching: stitch images in pairs, then recursively stitch the results.
    """
    intermediate_images = []

    # Step 1: Stitch in pairs
    for i in range(start, end + 1,2):
        image1 = f"{input_dir}/{i}.bmp"
        image2 = f"{input_dir}/{i+1}.bmp"
        pair_file = f"{input_dir}/pair_{i}_{i+1}.txt"
        npz_file = f"{output_dir}/{i}_{i+1}_matches.npz"
        stitched_output = f"{input_dir}/s_{i}_{i+1}.bmp"

        # Write the pair file
        with open(pair_file, 'w') as f:
            f.write(f"{i}.bmp {i+1}.bmp")

        # Run SuperGlue for the pair
        run_superglue(script_path, pair_file, input_dir, output_dir)

        # Stitch the pair
        stitched_image = stitch_images(image1, image2, npz_file, stitched_output)
        intermediate_images.append(stitched_image)
        print("stitching", i , i+1)

    # Step 2: Recursively stitch intermediate results
    while len(intermediate_images) > 1:
        next_level_images = []
        for i in range(0, len(intermediate_images),2):
            if i + 1 >= len(intermediate_images):
                # If odd number of images, carry the last one to the next level
                next_level_images.append(intermediate_images[i])
                continue

            image1 = intermediate_images[i]
            image2 = intermediate_images[i + 1]
            npz_file = f"{output_dir}/{Path(image1).stem}_{Path(image2).stem}_matches.npz"
            pair_file = f"{input_dir}/pair_{Path(image1).stem}_{Path(image2).stem}.txt"
            stitched_output = f"{input_dir}/s_{Path(image1).stem}_{Path(image2).stem}.bmp"

            # Write the pair file
            with open(pair_file, 'w') as f:
                f.write(f"{Path(image1).name} {Path(image2).name}")
            print("stitching", npz_file)
            # Run SuperGlue for the pair
            run_superglue(script_path, pair_file, input_dir, output_dir)

            # Stitch the pair
            stitched_image = stitch_images(image1, image2, npz_file, stitched_output)
            next_level_images.append(stitched_image)

        intermediate_images = next_level_images

    # Final result
    return intermediate_images[0] if intermediate_images else None

# Example usage
final_panorama = hierarchical_stitching(
    start=3, 
    end=20, 
    script_path="match_pairs.py", 
    input_dir="Stitch_with_perspective/input", 
    output_dir="Stitch_with_perspective/output"
)
print(f"Final panorama saved at: {final_panorama}")


stitching 3 4
stitching 5 6
stitching 7 8
stitching 9 10
stitching 11 12
stitching 13 14
stitching 15 16
stitching 17 18
stitching 19 20
stitching Stitch_with_perspective/output/s_3_4_s_5_6_matches.npz
stitching Stitch_with_perspective/output/s_7_8_s_9_10_matches.npz
stitching Stitch_with_perspective/output/s_11_12_s_13_14_matches.npz
stitching Stitch_with_perspective/output/s_15_16_s_17_18_matches.npz
stitching Stitch_with_perspective/output/s_s_3_4_s_5_6_s_s_7_8_s_9_10_matches.npz
stitching Stitch_with_perspective/output/s_s_11_12_s_13_14_s_s_15_16_s_17_18_matches.npz
stitching Stitch_with_perspective/output/s_s_s_3_4_s_5_6_s_s_7_8_s_9_10_s_s_s_11_12_s_13_14_s_s_15_16_s_17_18_matches.npz
stitching Stitch_with_perspective/output/s_s_s_s_3_4_s_5_6_s_s_7_8_s_9_10_s_s_s_11_12_s_13_14_s_s_15_16_s_17_18_s_19_20_matches.npz
Final panorama saved at: Stitch_with_perspective/input/s_s_s_s_s_3_4_s_5_6_s_s_7_8_s_9_10_s_s_s_11_12_s_13_14_s_s_15_16_s_17_18_s_19_20.bmp


In [6]:
import cv2 as cv
import numpy as np
import subprocess
from pathlib import Path
import matplotlib.pyplot as plt

def run_superglue(script_path, input_pairs, input_dir, output_dir, superglue_weights="outdoor", resize_dimensions=-1):
    """
    Runs the SuperGlue matching script with specified parameters.
    """
    command = [
        "python3", script_path,
        "--input_pairs", input_pairs,
        "--input_dir", input_dir,
        "--output_dir", output_dir,
        "--superglue", superglue_weights,
        "--viz",
        "--resize", str(resize_dimensions),
        "--max_keypoints", "2048",
        "--nms_radius", "5",
        "--keypoint_threshold", "0.05",
        "--match_threshold", "0.9",
        "--resize_float"
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print("Error running SuperGlue:")
        print(result.stderr)
    return result

def loadNPZ(npz_file):
    """
    Loads the points from an NPZ file generated by SuperGlue.
    """
    data = np.load(npz_file)
    keypoints0 = data['keypoints0']
    keypoints1 = data['keypoints1']
    matches = data['matches']
    valid = matches > -1
    points1 = keypoints0[valid]
    points2 = keypoints1[matches[valid]]
    return points1, points2

def stitch_images(image1_path, image2_path, npz_path, output_path):
    """
    Stitches two images together using points from a SuperGlue-generated NPZ file.
    """
    im_left = cv.imread(image1_path, cv.IMREAD_ANYCOLOR)
    im_right = cv.imread(image2_path, cv.IMREAD_ANYCOLOR)
    point_set1, point_set2 = loadNPZ(npz_path)
    H, status = cv.findHomography(point_set1, point_set2, cv.RANSAC, 5.0)
    rows1, cols1 = im_right.shape[:2]
    rows2, cols2 = im_left.shape[:2]
    points1 = np.float32([[0, 0], [0, rows1], [cols1, rows1], [cols1, 0]]).reshape(-1, 1, 2)
    points = np.float32([[0, 0], [0, rows2], [cols2, rows2], [cols2, 0]]).reshape(-1, 1, 2)
    points2 = cv.perspectiveTransform(points, H)
    points2_list = [point[0].tolist() for point in points2]
    list_of_points = np.concatenate((points1, points2), axis=0)

    [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() )
    [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() )

    H_translation = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]).dot(H)

    output_img = cv.warpPerspective(im_left, H_translation, (x_max - x_min, y_max - y_min))
    output_img[-y_min:rows1 + (-y_min), -x_min:cols1 + (-x_min)] = im_right
    input_pts = np.float32([[0,0], points2_list[1], points2_list[2], [cols1, 0]])
    output_pts = np.float32([[0, 0],
                        [0, points2_list[1][1]],
                        points2_list[2],
                        [points2_list[2][0], 0]])
    # Compute the perspective transform M
    M = cv.getPerspectiveTransform(input_pts, output_pts)
    final_img = cv.warpPerspective(output_img, M, (x_max - x_min, y_max - y_min))
    cv.imwrite(output_path, final_img)
    return output_path

def hierarchical_stitching(start, end, script_path, input_dir, output_dir):
    """
    Hierarchical stitching: stitch images in pairs, then recursively stitch the results.
    """
    intermediate_images = []

    # Step 1: Stitch in pairs
    for i in range(start, end + 1,2):
        image1 = f"{input_dir}/{i}.jpg"
        image2 = f"{input_dir}/{i+1}.jpg"
        pair_file = f"{input_dir}/pair_{i}_{i+1}.txt"
        npz_file = f"{output_dir}/{i}_{i+1}_matches.npz"
        stitched_output = f"{input_dir}/{i}_{i+1}.jpg"

        # Write the pair file
        with open(pair_file, 'w') as f:
            f.write(f"{i}.jpg {i+1}.jpg")

        # Run SuperGlue for the pair
        run_superglue(script_path, pair_file, input_dir, output_dir)

        # Stitch the pair
        stitched_image = stitch_images(image1, image2, npz_file, stitched_output)
        intermediate_images.append(stitched_image)
        print("stitching", i , i+1)

    # Step 2: Recursively stitch intermediate results
    while len(intermediate_images) > 1:
        next_level_images = []
        for i in range(0, len(intermediate_images),2):
            if i + 1 >= len(intermediate_images):
                # If odd number of images, carry the last one to the next level
                next_level_images.append(intermediate_images[i])
                continue

            image1 = intermediate_images[i]
            image2 = intermediate_images[i + 1]
            npz_file = f"{output_dir}/{Path(image1).stem}_{Path(image2).stem}_matches.npz"
            pair_file = f"{input_dir}/pair_{Path(image1).stem}_{Path(image2).stem}.txt"
            stitched_output = f"{input_dir}/{Path(image1).stem}_{Path(image2).stem}.jpg"

            # Write the pair file
            with open(pair_file, 'w') as f:
                f.write(f"{Path(image1).name} {Path(image2).name}")
            print("stitching", npz_file)
            # Run SuperGlue for the pair
            run_superglue(script_path, pair_file, input_dir, output_dir)

            # Stitch the pair
            stitched_image = stitch_images(image1, image2, npz_file, stitched_output)
            next_level_images.append(stitched_image)

        intermediate_images = next_level_images

    # Final result
    return intermediate_images[0] if intermediate_images else None

# Example usage
final_panorama = hierarchical_stitching(
    start=1, 
    end=48, 
    script_path="match_pairs.py", 
    input_dir="Stitch_with_perspective/input_t", 
    output_dir="Stitch_with_perspective/output_t"
)
print(f"Final panorama saved at: {final_panorama}")


stitching 1 2
stitching 3 4
stitching 5 6
stitching 7 8
stitching 9 10
stitching 11 12
stitching 13 14
stitching 15 16
stitching 17 18
stitching 19 20
stitching 21 22
stitching 23 24
stitching 25 26
stitching 27 28
stitching 29 30
stitching 31 32
stitching 33 34
stitching 35 36
stitching 37 38
stitching 39 40
stitching 41 42
stitching 43 44
stitching 45 46
stitching 47 48
stitching Stitch_with_perspective/output_t/1_2_3_4_matches.npz
stitching Stitch_with_perspective/output_t/5_6_7_8_matches.npz
stitching Stitch_with_perspective/output_t/9_10_11_12_matches.npz
stitching Stitch_with_perspective/output_t/13_14_15_16_matches.npz
stitching Stitch_with_perspective/output_t/17_18_19_20_matches.npz
stitching Stitch_with_perspective/output_t/21_22_23_24_matches.npz
stitching Stitch_with_perspective/output_t/25_26_27_28_matches.npz
stitching Stitch_with_perspective/output_t/29_30_31_32_matches.npz
stitching Stitch_with_perspective/output_t/33_34_35_36_matches.npz
stitching Stitch_with_perspecti

FileNotFoundError: [Errno 2] No such file or directory: 'Stitch_with_perspective/input_t/pair_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20_21_22_23_24_25_26_27_28_29_30_31_32_33_34_35_36_37_38_39_40_41_42_43_44_45_46_47_48.txt'